In [44]:
import pandas as pd
import numpy as np
import difflib as dl
import re
import requests
from io import StringIO
from tqdm import tqdm


def getCurie(name):
    try:
        itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=ChemicalOrDrugOrTreatment'
        returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
        resolvedName = returned.curie
        resolvedLabel = returned.label
        return resolvedName, resolvedLabel
    except:
        print("name resolver error for ", name)
        return ["Error"], ["Error"]

grasData = pd.read_csv("GRASNotices.csv", skiprows=[0,1], encoding='latin-1')
Opinions = grasData["FDA\'s Letter"]
indices = list('FDA has no questions' in i for i in Opinions)
approved_gras_substances = grasData[indices]
substanceList = approved_gras_substances['Substance']


approved_usa = []
available_usa = []
curie = []
label = []
therapy_name = []

for i in tqdm(substanceList, total=len(substanceList)):
    i = i.replace("<i>", "").replace("</i>", "")
    therapy_name.append(i)
    approved_usa.append("True")
    available_usa.append("GRAS")
    curies, labels = getCurie(i)
    curie.append(curies[0])
    label.append(labels[0])

data = pd.DataFrame({
    "therapy_name":therapy_name,
    "approved_usa":approved_usa,
    "available_usa":available_usa,
    "Single ID": curie,
    "ID Label": label
})

data.to_excel("gras_list.xlsx")




 60%|████████████████████████▋                | 535/888 [08:01<02:16,  2.59it/s]

name resolver error for   Algal fat derived from <I>Prototheca moriformis</I> (S7737)


 98%|████████████████████████████████████████▎| 872/888 [12:50<00:06,  2.65it/s]

name resolver error for   Anaerobutyricum soehngenii CH106


100%|█████████████████████████████████████████| 888/888 [12:59<00:00,  1.14it/s]


In [45]:
print(data)

                                          therapy_name approved_usa  \
0     Solin oil (low linolenic acid flaxseed oil or...         True   
1                                     Sodium bisulfate         True   
2     Transglutaminase from Streptoverticillium mob...         True   
3                        Dioctyl sodium sulfosuccinate         True   
4     Pectin esterase derived from Aspergillus oryz...         True   
..                                                 ...          ...   
883          Pea protein fermented by shiitake mycelia         True   
884                                     Monoglycerides         True   
885                         Hydrolyzed poultry protein         True   
886   Nuclease enzyme preparation produced by Bacil...         True   
887   Chitosan and beta-1,3-glucans from white butt...         True   

    available_usa                  Single ID  \
0            GRAS           DRUGBANK:DB11357   
1            GRAS              UMLS:C0889164   
2  

In [46]:
data.to_excel("gras_list.xlsx")